In [1]:
import pandas as pd
import csv
data1 = pd.read_csv("/home/jeehyun/Desktop/p2/telscope-log/AWS_dataset/unstructured/AWS_abnormal_log.csv", sep=",")
data2 = pd.read_csv("/home/jeehyun/Desktop/p2/telscope-log/AWS_dataset/unstructured/AWS_normal_log.csv", sep=",")
data1.to_pickle('aws_data1.pickle')
data2.to_pickle('aws_data2.pickle')

In [3]:
import os
import csv
import copy
import pickle
import gensim
import numpy as np

with open('aws_data1.pickle','rb') as handler:
    adv_list1 = pickle.load(handler)
with open('aws_data2.pickle','rb') as handler:
    adv_list2 = pickle.load(handler)
#print(adv_list1.columns)
#print(type(adv_list1))
#print(len(adv_list1))
#adv_list = adv_list.astype({'Occurrences' : 'string'})
#adv_list['result'] = adv_list['EventId'] + " " + adv_list['EventTemplate'] + " " + adv_list['Occurrences']
adv_list1['result'] = adv_list1['Content']
adv_list2['result'] = adv_list2['Content']
# a corpus = a list of sentences
# a sentence = a list of words
corpus = []
abnormal = []
normal = []
for i in range(len(adv_list1['result'])):
    corpus.append(adv_list1['result'].loc[i].split(" "))
    abnormal.append(adv_list1['result'].loc[i].split(" "))
for i in range(len(adv_list2['result'])):
    corpus.append(adv_list2['result'].loc[i].split(" "))
    normal.append(adv_list2['result'].loc[i].split(" "))
#corpus = adv_list.to_numpy()
print(corpus[0])

['jeehyun-main.link', 'b344fadcff6fe266163f3de3e1b3979e6d5938054fef6809556a6057717cdc71', '72.198.67.233', 'arn:aws:iam::179704931439:user/dhwlgus3456', '-', 'REST.PUT.OBJECT', '1', 'PUT,/jeehyun-main.link/password.txt,HTTP/1.1', '200', '-', '-', '-', '-', '-', '-', 'aws-cli/1.22.34,Python/3.10.6,Linux/5.15.0-60-generic,botocore/1.29.129', '-', '-', 'SigV4', 'ECDHE-RSA-AES128-GCM-SHA256', 'AuthHeader', 's3.amazonaws.com', 'TLSv1.2', '-', '-']


In [4]:
# FastText model is also provided by gensim package
# It is an advanced word2vec, see https://radimrehurek.com/gensim/models/fasttext.html

# Generate a dictionary for every word (a mapping between a word and its vector representation)
w2v_model_50 = gensim.models.Word2Vec(corpus, vector_size=50, min_count=1, workers=2, epochs=100)

## Save the dictionary if you want to re-use it
#w2v_model_50.save("patch_commit_voc50_mc5.w2v")

## If you have a saved dictionary, you can load it (so you don't need to repeatedly train/build-up it)
#  w2v_model_50 = gensim.models.Word2Vec.load("patch_commit_voc50_mc5.w2v")

print(w2v_model_50.wv.vectors.shape)

## Print vocabulary size, i.e. the # of words in the dictionary
vocab_size = len(w2v_model_50.wv.key_to_index)
print(vocab_size)

(122, 50)
122


In [45]:
# save model
#w2v_model_50.save("patch_commit_voc50_mc5.w2v")

In [5]:
# if you have a 'word', and you want to get index, you can use
discard_idx = w2v_model_50.wv.key_to_index
print(discard_idx)
print("\n")
# if you want to get the vector representation for a give word
print(w2v_model_50.wv[0])
type(w2v_model_50.wv)

{'-': 0, 'jeehyun-main.link': 1, 'b344fadcff6fe266163f3de3e1b3979e6d5938054fef6809556a6057717cdc71': 2, '72.198.67.233': 3, 'AuthHeader': 4, 'SigV4': 5, '1': 6, 'TLSv1.2': 7, 'ECDHE-RSA-AES128-GCM-SHA256': 8, 'arn:aws:iam::179704931439:user/dhwlgus3456': 9, 's3.amazonaws.com': 10, '200': 11, 'aws-cli/1.22.34,Python/3.10.6,Linux/5.15.0-60-generic,botocore/1.29.129': 12, 'REST.PUT.OBJECT': 13, '204': 14, 'DELETE,/jeehyun-main.link/user.passwd,HTTP/1.1': 15, 'PUT,/jeehyun-main.link/password.txt,HTTP/1.1': 16, 'REST.DELETE.OBJECT': 17, '0': 18, 'Mozilla/5.0,(Windows,NT,10.0;,Win64;,x64),AppleWebKit/537.36,(KHTML,,like,Gecko),Chrome/113.0.0.0,Safari/537.36': 19, 'WEBSITE.GET.OBJECT': 20, 'S3Console/0.4,cfg/retry-mode/legacy': 21, '304': 22, 's3.us-east-1.amazonaws.com': 23, 'svc:s3.amazonaws.com': 24, 'http://jeehyun-main.link/index.html': 25, 'http://jeehyun-main.link/': 26, 'jeehyun-main.link.s3.us-east-1.amazonaws.com': 27, 'Boto3/1.26.129,Python/3.10.6,Linux/5.15.0-60-generic,Botocore/1

gensim.models.keyedvectors.KeyedVectors

In [6]:
import json
dictionary = {}
for i in range(len(w2v_model_50.wv)):
    dictionary[i] = w2v_model_50.wv[i].tolist()
    
a = json.dumps(dictionary,indent=4)

with open("AWS_vec.json", 'w') as f:
    f.write(a)

In [7]:
for i in range(len(corpus)):
    for j in range(len(corpus[i])):
        corpus[i][j] = discard_idx.get(corpus[i][j])
for i in range(len(normal)):
    for j in range(len(normal[i])):
        normal[i][j] = discard_idx.get(normal[i][j])
for i in range(len(abnormal)):
    for j in range(len(abnormal[i])):
        abnormal[i][j] = discard_idx.get(abnormal[i][j])

In [9]:
normal

[[1,
  2,
  3,
  0,
  0,
  20,
  6,
  68,
  11,
  0,
  0,
  0,
  0,
  0,
  36,
  19,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0],
 [1,
  2,
  3,
  9,
  0,
  63,
  18,
  37,
  11,
  0,
  0,
  0,
  0,
  0,
  0,
  21,
  0,
  0,
  5,
  8,
  4,
  10,
  7,
  0,
  0],
 [1,
  2,
  3,
  9,
  0,
  49,
  18,
  48,
  11,
  0,
  0,
  0,
  0,
  0,
  0,
  21,
  0,
  0,
  5,
  8,
  4,
  27,
  7,
  0,
  0],
 [1,
  2,
  3,
  9,
  0,
  30,
  18,
  45,
  11,
  0,
  0,
  0,
  0,
  0,
  0,
  21,
  0,
  0,
  5,
  8,
  4,
  27,
  7,
  0,
  0],
 [1,
  2,
  3,
  9,
  0,
  43,
  18,
  42,
  11,
  0,
  0,
  0,
  0,
  0,
  0,
  21,
  0,
  0,
  5,
  8,
  4,
  27,
  7,
  0,
  0],
 [1,
  2,
  3,
  0,
  0,
  20,
  6,
  72,
  22,
  0,
  0,
  0,
  0,
  0,
  26,
  19,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0],
 [1,
  2,
  3,
  0,
  0,
  20,
  6,
  71,
  11,
  0,
  0,
  0,
  0,
  0,
  26,
  19,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0],
 [1,
  2,
  3,
  0,
  0,
  20,
  6,
  70,
  11,
  0,
  0,
  0,
  0,
  0,
  26,


In [10]:
pd.DataFrame(corpus).to_csv('indexed_all.txt', sep=' ',header=False, index=False)
pd.DataFrame(normal).to_csv('indexed_normal.txt', sep=' ',header=False, index=False)
pd.DataFrame(abnormal).to_csv('indexed_abnormal.txt', sep=' ',header=False, index=False)

In [15]:
json_string = "{\n\t"
try:
    for value in w2v_model_50.wv:
        for key in discard_idx:
            json_string += "\""+ str(discard_idx[key]) + "\" : [\n\t"
            for element in value:
                json_string += "\t" + str(element) + ",\n\t"
            if(65json.... != discard_idx[key]):
                json_string += "],\n\t"
            else:
                json_string += "]\n"
        json_string +="}"
        break
except:
    pass

In [ ]:
import json

In [16]:
maxlen = 100 # we allow at most 100 words for each sentence (a sentence is a sequence of words)

def generate_list(sentence, word2vecs, max_len):
    
    result_list = np.zeros((max_len), dtype = np.int32)
    
    i = 0
    
    for word in sentence:
        
        if word in word2vecs.key_to_index.keys():
            # print(word)
            # print(word2vecs.key_to_index[word])
            result_list[i] = word2vecs.key_to_index[word]
        #else:
            ## if the word cannot be found, use a default value
        i += 1
    return result_list


a_list_representation_for_a_sentence = generate_list(adv_list[0], w2v_model_50.wv, maxlen)

# the representation for a sentence is a list, each word is represented by it index in dictionary
print(a_list_representation_for_a_sentence.shape)

(100,)


In [17]:
print(a_list_representation_for_a_sentence)

[1094 2215 1270   66   29    2 2215   14  237  793    1  804    5    8
  520   64  102    0    8 2215  759   16 1688    0  324  169    2  245
  526    0    1    7   32  692 1684  245  813  102   31    2   62   15
    2 2215 1270   73  708    2  557  324    5   50  550  813   79   24
   79   18    5 1485  105  257   76    9    1    6  176   20    0    0
    1    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0]
